In [136]:
import os
import glob
import re
import numpy as np
import numpy.fft as fft
import mne
import matplotlib.pyplot as plt
from scipy import stats, signal
from numpy import save, load
import pandas as pd
import sklearn
from sklearn.svm import SVC

In [137]:
np.arange(1, 10)

array([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [138]:
def read_patient(subject_id):
    # subject_id=1
    base_path = "min10/"
    X=load('min10/data_chb01/features_{}_00.npy'.format(subject_id))
    y=load('min10/data_chb01/targets_{}_00.npy'.format(subject_id))
    edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb{:02d}/*.npy".format(subject_id))))
    files=len(edf_file_names)
    print(files)
    print(X.shape)
    start=1
    for fileno in range(start, files//2):
        print(fileno)
        X=np.concatenate((X, load('min10/data_chb{:02d}/features_{}_{:02d}.npy'.format(subject_id, subject_id,   fileno))))
        y=np.concatenate((y, load('min10/data_chb{:02d}/targets_{}_{:02d}.npy'.format(subject_id,subject_id,     fileno))))
    return X,y


In [139]:
X,y=read_patients(2,3, 'features/')

72
(408, 208)


In [140]:
X.shape, y.shape

((13018, 208), (13018,))

In [141]:
channel_order= {'FP1-F7':0, 'F7-T7':1, 'T7-P7':2, 'P7-O1':3, 'FP1-F3':4, 'F3-C3':5, 'C3-P3':6, 'P3-O1':7, 'FP2-F4':8, 'F4-C4':9, 'C4-P4':10, 'P4-O2':11, 'FP2-F8':12, 'F8-T8':13, 'T8-P8-0':14, 'P8-O2':15, 'FZ-CZ':16, 'CZ-PZ':17, 'P7-T7':18, 'T7-FT9':19, 'FT9-FT10':20, 'FT10-T8':21 ,'T8-P8-1':22 }
col_names=[]
for ch in channel_order:
    # print(ch)
    for i in range(0,9):
        col_names.append(str(ch)+'-'+str(i))
col_names.append('patient')
print(col_names)

['FP1-F7-0', 'FP1-F7-1', 'FP1-F7-2', 'FP1-F7-3', 'FP1-F7-4', 'FP1-F7-5', 'FP1-F7-6', 'FP1-F7-7', 'FP1-F7-8', 'F7-T7-0', 'F7-T7-1', 'F7-T7-2', 'F7-T7-3', 'F7-T7-4', 'F7-T7-5', 'F7-T7-6', 'F7-T7-7', 'F7-T7-8', 'T7-P7-0', 'T7-P7-1', 'T7-P7-2', 'T7-P7-3', 'T7-P7-4', 'T7-P7-5', 'T7-P7-6', 'T7-P7-7', 'T7-P7-8', 'P7-O1-0', 'P7-O1-1', 'P7-O1-2', 'P7-O1-3', 'P7-O1-4', 'P7-O1-5', 'P7-O1-6', 'P7-O1-7', 'P7-O1-8', 'FP1-F3-0', 'FP1-F3-1', 'FP1-F3-2', 'FP1-F3-3', 'FP1-F3-4', 'FP1-F3-5', 'FP1-F3-6', 'FP1-F3-7', 'FP1-F3-8', 'F3-C3-0', 'F3-C3-1', 'F3-C3-2', 'F3-C3-3', 'F3-C3-4', 'F3-C3-5', 'F3-C3-6', 'F3-C3-7', 'F3-C3-8', 'C3-P3-0', 'C3-P3-1', 'C3-P3-2', 'C3-P3-3', 'C3-P3-4', 'C3-P3-5', 'C3-P3-6', 'C3-P3-7', 'C3-P3-8', 'P3-O1-0', 'P3-O1-1', 'P3-O1-2', 'P3-O1-3', 'P3-O1-4', 'P3-O1-5', 'P3-O1-6', 'P3-O1-7', 'P3-O1-8', 'FP2-F4-0', 'FP2-F4-1', 'FP2-F4-2', 'FP2-F4-3', 'FP2-F4-4', 'FP2-F4-5', 'FP2-F4-6', 'FP2-F4-7', 'FP2-F4-8', 'F4-C4-0', 'F4-C4-1', 'F4-C4-2', 'F4-C4-3', 'F4-C4-4', 'F4-C4-5', 'F4-C4-6', 'F4-

In [142]:
df=pd.DataFrame(data=X, columns=col_names)
df['target']=y
df=df.dropna()
df.shape

(13018, 209)

In [143]:
df=df[(df['target']!=3)]

In [144]:
from scipy.stats.stats import pearsonr
coefs=np.zeros(207)
p_vals=np.zeros(207)
for i in range(0,207):
    (coefs[i], p_vals[i])=pearsonr(df[col_names[i]], df['target'])

In [145]:
np.argmax(coefs), np.argmin(p_vals)

(107, 6)

In [146]:
corr_df=pd.DataFrame(data={'corr01':coefs,'p01': p_vals})
corr_df['feature']=col_names[:-1]
corr_df[corr_df['p01']<0.05]

,corr01,p01,feature
0,-0.128907,7.740412e-48,FP1-F7-0
1,-0.173240,1.708974e-85,FP1-F7-1
2,-0.107872,6.151836e-34,FP1-F7-2
3,-0.143646,4.381435e-59,FP1-F7-3
5,-0.044234,6.772802e-07,FP1-F7-5
...,...,...,...
202,0.055899,3.392034e-10,T8-P8-1-4
203,-0.052819,2.971652e-09,T8-P8-1-5
204,0.246336,1.471067e-173,T8-P8-1-6
205,0.263578,2.547763e-199,T8-P8-1-7


In [147]:
corr_df.nlargest(40, ['corr01'])

,corr01,p01,feature
107,0.610408,0.000000e+00,P4-O2-8
35,0.595723,0.000000e+00,P7-O1-8
98,0.594710,0.000000e+00,C4-P4-8
143,0.573800,0.000000e+00,P8-O2-8
116,0.571192,0.000000e+00,FP2-F8-8
179,0.568081,0.000000e+00,T7-FT9-8
44,0.533156,0.000000e+00,FP1-F3-8
53,0.530325,0.000000e+00,F3-C3-8
188,0.503084,0.000000e+00,FT9-FT10-8
8,0.475325,0.000000e+00,FP1-F7-8


In [148]:
corr_df.nsmallest(30, ['corr01'])

,corr01,p01,feature
156,-0.259488,4.975209e-193,CZ-PZ-3
153,-0.234823,1.896826e-157,CZ-PZ-0
147,-0.218445,5.288289e-136,FZ-CZ-3
57,-0.214620,3.068379e-131,C3-P3-3
163,-0.205764,1.455308e-120,P7-T7-1
19,-0.205615,2.182816e-120,T7-P7-1
37,-0.193911,4.916354e-107,FP1-F3-1
48,-0.190158,6.228862e-103,F3-C3-3
144,-0.189315,5.066851e-102,FZ-CZ-0
54,-0.187269,7.859195e-100,C3-P3-0


In [149]:
corr_df=corr_df.sort_values('corr01', ascending=False)
corr_df

,corr01,p01,feature
107,0.610408,0.000000e+00,P4-O2-8
35,0.595723,0.000000e+00,P7-O1-8
98,0.594710,0.000000e+00,C4-P4-8
143,0.573800,0.000000e+00,P8-O2-8
116,0.571192,0.000000e+00,FP2-F8-8
...,...,...,...
163,-0.205764,1.455308e-120,P7-T7-1
57,-0.214620,3.068379e-131,C3-P3-3
147,-0.218445,5.288289e-136,FZ-CZ-3
153,-0.234823,1.896826e-157,CZ-PZ-0


In [ ]:
grid_df.to_csv(r'grid_search.csv')

In [84]:
corr_df['feature']

47      F3-C3-2
65      P3-O1-2
29      P7-O1-2
48      F3-C3-3
164     P7-T7-2
         ...   
78     FP2-F4-6
80     FP2-F4-8
2      FP1-F7-2
77     FP2-F4-5
76     FP2-F4-4
Name: feature, Length: 207, dtype: object

In [85]:
df=df.reindex(columns=corr_df['feature'])
df


feature,F3-C3-2,P3-O1-2,P7-O1-2,F3-C3-3,P7-T7-2,T7-P7-2,P3-O1-3,FZ-CZ-2,F8-T8-2,P7-T7-3,...,FP1-F7-3,CZ-PZ-8,CZ-PZ-4,FP1-F3-4,FP2-F4-1,FP2-F4-6,FP2-F4-8,FP1-F7-2,FP2-F4-5,FP2-F4-4
0,0.930633,0.822941,0.565681,1.975643,0.767464,0.767464,2.020501,0.922018,2.593229,2.026237,...,11.113483,30.212417,9.509009,460.576569,185.230988,153.009369,834.308265,10.275364,42.965191,406.118134
1,0.944644,0.822464,0.588373,2.256453,0.817389,0.817389,2.099589,1.082254,2.013219,2.162400,...,11.635523,26.164239,7.921816,517.882019,184.683868,159.239899,916.972032,10.569725,44.596828,475.158691
2,1.005725,0.755330,0.693867,2.537391,0.795642,0.795642,2.017942,1.084726,1.367920,1.804190,...,9.847975,21.692061,6.137328,320.266724,108.013680,83.900970,533.061094,9.570168,24.854679,279.845703
3,0.897233,0.734125,0.621030,2.070829,0.664712,0.664712,2.203984,0.838089,1.643205,1.516540,...,7.499085,19.284328,5.126681,187.052170,91.145386,64.401649,355.087003,6.070765,16.468584,158.311935
4,1.167659,0.725621,0.643493,1.701107,1.239763,1.239763,2.198303,0.989205,3.611138,2.524845,...,12.209982,36.224937,10.890305,138.007599,49.599998,42.035198,234.592996,11.789531,10.751387,103.478966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14572,0.682462,0.697292,0.550103,1.621111,0.507717,0.507717,1.979153,1.042357,0.885445,1.796431,...,1.927700,967.407725,112.776039,7.420779,7.059189,38.153008,73.124627,2.425752,5.873767,15.246887
14573,0.694258,0.659297,0.548741,1.733127,0.571982,0.571982,1.933833,1.001961,0.851418,1.770358,...,1.300381,1055.595557,127.472321,9.705343,7.657333,39.721512,75.633780,0.536385,6.216688,16.398058
14574,0.754740,0.725180,0.647252,1.814148,0.601103,0.601103,1.809154,0.801546,0.719815,1.444173,...,1.671109,986.620917,139.782654,21.472988,9.820520,38.469189,83.369867,0.642153,6.075457,22.542147
14575,0.680372,0.791564,0.697842,1.803425,0.562883,0.562883,2.216992,0.802544,0.483032,1.205672,...,1.881431,1073.768412,103.645889,22.058796,11.026934,43.046490,90.274166,0.651625,7.100677,22.456596


In [105]:
from scipy.stats.stats import pearsonr

def order_correlation(data, target):
    df=pd.DataFrame(data=X, columns=col_names)
    df['target']=target
    df=df.dropna()
    df=df[(df['target']!=3)]
    coefs=np.zeros(len(df.columns)-2)
    p_vals=np.zeros(len(df.columns)-2)
    for i in range(0,len(df.columns)-2):
        (coefs[i], p_vals[i])=pearsonr(df[df.columns[i]], df['target'])
    corr_df=pd.DataFrame(data={'corr01':coefs,'p01': p_vals})
    corr_df['feature']=df.columns[:-2]
    corr_df=corr_df.sort_values('corr01', ascending=False)
    return np.append(corr_df['feature'], 'patient')

    


In [106]:
ft=order_correlation(X, y)
ft

array(['F3-C3-2', 'P3-O1-2', 'P7-O1-2', 'F3-C3-3', 'P7-T7-2', 'T7-P7-2',
       'P3-O1-3', 'FZ-CZ-2', 'F8-T8-2', 'P7-T7-3', 'F8-T8-3', 'T7-P7-3',
       'P7-O1-3', 'FP2-F4-0', 'FT9-FT10-3', 'F7-T7-2', 'FP1-F3-0',
       'C3-P3-2', 'F4-C4-2', 'T7-FT9-2', 'F3-C3-0', 'FZ-CZ-3', 'C4-P4-2',
       'FZ-CZ-0', 'FT9-FT10-2', 'F7-T7-3', 'P4-O2-2', 'FT10-T8-2',
       'F8-T8-0', 'C3-P3-3', 'T7-FT9-3', 'FP2-F8-0', 'P4-O2-3',
       'FP2-F4-3', 'CZ-PZ-2', 'F4-C4-3', 'FP1-F3-3', 'T8-P8-1-2',
       'T8-P8-0-2', 'F4-C4-0', 'FT10-T8-3', 'T8-P8-1-3', 'T8-P8-0-3',
       'C4-P4-3', 'C3-P3-0', 'FP1-F7-0', 'P3-O1-0', 'P8-O2-2',
       'FT9-FT10-0', 'C4-P4-0', 'F7-T7-0', 'FP2-F8-3', 'P4-O2-0',
       'P8-O2-3', 'CZ-PZ-0', 'CZ-PZ-3', 'P7-O1-0', 'P7-T7-0', 'T7-P7-0',
       'T8-P8-1-0', 'T8-P8-0-0', 'T7-FT9-0', 'FT10-T8-0', 'P7-O1-6',
       'P7-O1-8', 'P7-O1-5', 'P7-O1-7', 'FP1-F7-6', 'FP2-F8-6', 'P7-T7-6',
       'T7-P7-6', 'P7-O1-1', 'T7-FT9-6', 'P7-O1-4', 'P7-T7-5', 'T7-P7-5',
       'T7-FT9-5', 'F7-T7-

In [110]:
newdf=pd.DataFrame(data=X, columns=col_names)
newdf


0        0.930633
1        0.944644
2        1.005725
3        0.897233
4        1.167659
           ...   
14572    0.682462
14573    0.694258
14574    0.754740
14575    0.680372
14576    0.734146
Name: F3-C3-2, Length: 14577, dtype: float64

In [113]:
newdf=newdf.reindex(columns=ft)
newdf


,F3-C3-2,P3-O1-2,P7-O1-2,F3-C3-3,P7-T7-2,T7-P7-2,P3-O1-3,FZ-CZ-2,F8-T8-2,P7-T7-3,...,CZ-PZ-8,CZ-PZ-4,FP1-F3-4,FP2-F4-1,FP2-F4-6,FP2-F4-8,FP1-F7-2,FP2-F4-5,FP2-F4-4,patient
0,0.930633,0.822941,0.565681,1.975643,0.767464,0.767464,2.020501,0.922018,2.593229,2.026237,...,30.212417,9.509009,460.576569,185.230988,153.009369,834.308265,10.275364,42.965191,406.118134,1.0
1,0.944644,0.822464,0.588373,2.256453,0.817389,0.817389,2.099589,1.082254,2.013219,2.162400,...,26.164239,7.921816,517.882019,184.683868,159.239899,916.972032,10.569725,44.596828,475.158691,1.0
2,1.005725,0.755330,0.693867,2.537391,0.795642,0.795642,2.017942,1.084726,1.367920,1.804190,...,21.692061,6.137328,320.266724,108.013680,83.900970,533.061094,9.570168,24.854679,279.845703,1.0
3,0.897233,0.734125,0.621030,2.070829,0.664712,0.664712,2.203984,0.838089,1.643205,1.516540,...,19.284328,5.126681,187.052170,91.145386,64.401649,355.087003,6.070765,16.468584,158.311935,1.0
4,1.167659,0.725621,0.643493,1.701107,1.239763,1.239763,2.198303,0.989205,3.611138,2.524845,...,36.224937,10.890305,138.007599,49.599998,42.035198,234.592996,11.789531,10.751387,103.478966,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14572,0.682462,0.697292,0.550103,1.621111,0.507717,0.507717,1.979153,1.042357,0.885445,1.796431,...,967.407725,112.776039,7.420779,7.059189,38.153008,73.124627,2.425752,5.873767,15.246887,1.0
14573,0.694258,0.659297,0.548741,1.733127,0.571982,0.571982,1.933833,1.001961,0.851418,1.770358,...,1055.595557,127.472321,9.705343,7.657333,39.721512,75.633780,0.536385,6.216688,16.398058,1.0
14574,0.754740,0.725180,0.647252,1.814148,0.601103,0.601103,1.809154,0.801546,0.719815,1.444173,...,986.620917,139.782654,21.472988,9.820520,38.469189,83.369867,0.642153,6.075457,22.542147,1.0
14575,0.680372,0.791564,0.697842,1.803425,0.562883,0.562883,2.216992,0.802544,0.483032,1.205672,...,1073.768412,103.645889,22.058796,11.026934,43.046490,90.274166,0.651625,7.100677,22.456596,1.0


In [114]:
np.array(newdf)

array([[  0.93063259,   0.82294136,   0.56568092, ...,  42.96519089,
        406.11813354,   1.        ],
       [  0.94464397,   0.82246417,   0.58837289, ...,  44.59682846,
        475.15869141,   1.        ],
       [  1.00572491,   0.75532961,   0.69386739, ...,  24.85467911,
        279.84570312,   1.        ],
       ...,
       [  0.7547397 ,   0.72518027,   0.64725196, ...,   6.07545662,
         22.54214668,   1.        ],
       [  0.68037152,   0.79156435,   0.69784164, ...,   7.10067654,
         22.45659637,   1.        ],
       [  0.73414552,   0.67073917,   0.60644311, ...,   7.79735184,
         17.55945206,   1.        ]])

In [129]:
def read_patients(start_id, end_id, base_path):
    subject_id=1
    # base_path = "sum_only/"
    # edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb01/*.npy".format(subject_id))))
    # files=len(edf_file_names)
    X=load(base_path+'data_chb01/features_{}_00.npy'.format(subject_id))
    y=load(base_path+'data_chb01/targets_{}_00.npy'.format(subject_id))
    for subject_id in range(start_id, end_id):
        edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb{:02d}/*.npy".format(subject_id))))
        files=len(edf_file_names)
        print(files)
        print(X.shape)
        start=0
        if subject_id==1:
            start=1
        for fileno in range(start, files//2):
            X=np.concatenate((X, load(base_path+'data_chb{:02d}/features_{}_{:02d}.npy'.format(subject_id, subject_id, fileno))))
            y=np.concatenate((y, load(base_path+'data_chb{:02d}/targets_{}_{:02d}.npy'.format(subject_id,subject_id, fileno))))
    return X,y


In [149]:
X2, y2=read_patients(1,5)

84
(408, 3)
72
(14577, 3)
76
(27187, 3)
84
(40755, 3)


In [189]:
df2=pd.DataFrame(data=X2, columns=['sum', 'rms', 'patient'])
df2['target']=y2
df2=df2.dropna()
df2
df2=df2[(df2['target']!=3)]

In [190]:
# plt.scatter(df2.index, df2['sum'], c=df2['target'])

In [191]:
# from scipy.stats.stats import pearsonr
# coefs2=np.zeros(6)
# p_vals2=np.zeros(6)
# for i in range(0,6):
#     (coefs2[i], p_vals2[i])=pearsonr(df2[df2.columns[i]], df2['patient'])
# coefs2

IndexError: index 4 is out of bounds for axis 0 with size 4

In [164]:
# corr_df2=pd.DataFrame(data={'corr01':coefs2,'p01': p_vals2})
# corr_df2['feature']=df2.columns[:-2]
# corr_df2

In [165]:
# corr_df2=pd.DataFrame(data={'corr01':coefs2,'p01': p_vals2})
# corr_df2['feature']=df2.columns[:-2]
# corr_df2

In [192]:
mean_sums=[]
df["mean_sum"]=0
for patient in pd.unique(df2['patient']):
    # patdf=df[df[207]==patient]
    # df2["mean_sum"][df2['patient']==patient]=np.mean(df2['sum'][(df2['patient']==patient)&(df2['target']==0)])
    mean_sum=np.mean(df2['sum'][(df2['patient']==patient)&(df2['target']==0)])
    # df2.where(df2['patient']!=patient, mean_sums[int(patient)])
    mean_sums.append(mean_sum)
print(mean_sums)

[341725.9808489427, 330571.7973419133, 183551.2078667481, 32665309.98580303]


In [193]:
df2

,sum,rms,patient,target
0,1.220569e+05,7.405896,1.0,0
1,1.198049e+05,7.694022,1.0,0
2,8.168956e+04,6.464692,1.0,0
3,6.076234e+04,6.407615,1.0,0
4,4.856459e+04,6.165314,1.0,0
...,...,...,...,...
97778,2.012290e+05,9.106253,4.0,0
97779,1.079467e+05,4.606265,4.0,0
97780,1.853029e+05,7.797244,4.0,0
97781,3.900796e+05,15.548586,4.0,0


In [194]:
  df2['new']=0
for patient in pd.unique(df2['patient']):
    df2['new'].where(df2['patient']!=patient, mean_sums[int(patient-1)], inplace=True)

    # df2['new']=df2['sum']/df2["mean_sum"]

df2=df2.reindex(columns= ['sum', 'rms', 'patient', 'new', 'target'])
df2

,sum,rms,patient,new,target
0,1.220569e+05,7.405896,1.0,3.417260e+05,0
1,1.198049e+05,7.694022,1.0,3.417260e+05,0
2,8.168956e+04,6.464692,1.0,3.417260e+05,0
3,6.076234e+04,6.407615,1.0,3.417260e+05,0
4,4.856459e+04,6.165314,1.0,3.417260e+05,0
...,...,...,...,...,...
97778,2.012290e+05,9.106253,4.0,3.266531e+07,0
97779,1.079467e+05,4.606265,4.0,3.266531e+07,0
97780,1.853029e+05,7.797244,4.0,3.266531e+07,0
97781,3.900796e+05,15.548586,4.0,3.266531e+07,0


In [195]:
from scipy.stats.stats import pearsonr
coefs2=np.zeros(4)
p_vals2=np.zeros(4)
# df21=df2[df2[pati]]
for i in range(0,4):
    (coefs2[i], p_vals2[i])=pearsonr(df2[df2.columns[i]], df2['patient'])
coefs2

array([0.03115819, 0.04265447, 1.        , 0.88707379])

In [ ]:
corr_df2=pd.DataFrame(data={'corr01':coefs2,'p01': p_vals2})
corr_df2['feature']=df2.columns[:-2]
corr_df2

In [206]:
a=np.array(df2.columns)
('target')

AttributeError: 'numpy.ndarray' object has no attribute 'remove'